#### （1）矢量化加速

In [2]:
import math
import time
import numpy as np
import torch
from d2l import torch as d2l

In [3]:
# 向量相加演示

n = 10000
a = torch.ones([n])
b = torch.ones([n])

In [4]:
class Timer: #@save
    """记录多次运行时间"""
    def __init__(self):
        self.times = []
        self.start() # 创建对象时开始计时

    def start(self):
        """启动计时器"""
        self.tik = time.time() # 记录开始时间

    def stop(self):
        """停止计时，并把时间记录到列表中"""
        self.times.append(time.time() - self.tik) # 当前时间 - 开始时间
        return self.times[-1]

    def avg(self):
        """记录平均时间"""
        return sum(self.times) / len(self.times)

    def sum(self):
        """记录总时间"""
        return sum(self.times)

    def cumsum(self):
        """返回累积时间"""
        return np.array(self.times).cumsum().tolist()
        
    

In [6]:
c = torch.zeros(n)
timer = Timer()
for i in range(n):
    c[i] = a[i] + b[i]
f'{timer.stop():.5f} sec'

'0.15981 sec'

In [7]:
timer.start()
d = a + b
f'{timer.stop():.5f} sec'

'0.00055 sec'

#### (2) 正态分布与平方损失